In [38]:
import pandas as pd
import numpy as np
import json

In [2]:
df_all = pd.read_csv('..\data\data_cleaned.csv')

In [3]:
df_all = df_all.replace({np.nan: 'None'})

In [4]:
df_all['id'] = df_all['id'].astype(str)

In [9]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 352 entries, 0 to 351
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   352 non-null    object
 1   equipment_type       352 non-null    object
 2   equipment_name       352 non-null    object
 3   maker                352 non-null    object
 4   type_of_maintenance  352 non-null    object
 5   frequency            352 non-null    object
 6   parts_required       352 non-null    object
dtypes: object(7)
memory usage: 19.4+ KB


In [10]:
df_all['equipment_type'].unique()

array(['Dishwasher', 'Refregerator'], dtype=object)

In [11]:
df_all.head()

,id,equipment_type,equipment_name,maker,type_of_maintenance,frequency,parts_required
0,0,Dishwasher,Series 6 SMS6ZCI48E,Bosch,Clean filter,Monthly,None
1,1,Dishwasher,WDT750SAKZ,Whirlpool,Descale,Quarterly,Descaling solution
2,2,Dishwasher,KDTM404KPS,KitchenAid,Check spray arms,Bi-monthly,None
3,3,Dishwasher,G 7316 SCU,Miele,Replace rinse aid,As needed,Rinse aid
4,4,Dishwasher,GDT665SSNSS,GE,Clean door seal,Monthly,Soft cloth


In [5]:
documents = df_all.to_dict(orient='records')

In [6]:
import minsearch

In [7]:
index = minsearch.Index(
    text_fields=['id','equipment_type', 'equipment_name', 'maker', 'type_of_maintenance', 'frequency',
       'parts_required'],
    keyword_fields=[id]
)

In [8]:
index.fit(documents)

In [111]:
question = "give me annual maintenance and required parts for Samsung Dishwasher"

In [112]:
boost = {'question':3.0, 'section':.5}
#knowledge_results = index.search(query=question, filter_dict={'equipment_name':'DW80R7061UG'},
#                       boost_dict=boost, num_results=5)
#knowledge_results = index.search(query=question
#                       boost_dict=boost, num_results=5)
documents = index.search(query=question, num_results=5)

In [113]:
documents

[{'id': '55',
  'equipment_type': 'Dishwasher',
  'equipment_name': 'DW80R5061US',
  'maker': 'Samsung',
  'type_of_maintenance': 'Check for error codes',
  'frequency': 'Monthly',
  'parts_required': 'None'},
 {'id': '135',
  'equipment_type': 'Dishwasher',
  'equipment_name': 'DW80R5060US',
  'maker': 'Samsung',
  'type_of_maintenance': 'Check for error codes',
  'frequency': 'Monthly',
  'parts_required': 'None'},
 {'id': '189',
  'equipment_type': 'Dishwasher',
  'equipment_name': 'DW80R5061US',
  'maker': 'Samsung',
  'type_of_maintenance': 'Inspect and clean door hinges',
  'frequency': 'Bi-annually',
  'parts_required': 'None'},
 {'id': '159',
  'equipment_type': 'Dishwasher',
  'equipment_name': 'DW80R7070UG',
  'maker': 'Samsung',
  'type_of_maintenance': 'Inspect and clean sump filter',
  'frequency': 'Monthly',
  'parts_required': 'None'},
 {'id': '179',
  'equipment_type': 'Dishwasher',
  'equipment_name': 'DW80K7050UG',
  'maker': 'Samsung',
  'type_of_maintenance': 'Clean

In [ ]:
equipment_name: {equipment_name},
maker: {maker},
type_of_maintenance: {type_of_maintenance},
frequency: {frequency},
parts_required: {parts_required}

In [59]:
df_all[df_all['maker'] == 'Samsung']['equipment_name'].unique()

array(['DW80R9950UG', 'DW80R2031US', 'DW80R7061UG', 'DW80R5060US',
       'DW80J3020US', 'DW80R5061US', 'DW80R9950UT', 'DW80R7061US',
       'DW80J7550US', 'DW80R7070UG', 'DW80R9350UG', 'DW80K7050UG',
       'DW80K5050US', 'RF23A9771SR', 'RF28R7351SR', 'RF23M8070SR',
       'RF28R6301SR', 'RF23A9771SG', 'RF28T5001SR', 'RF28R7351SG',
       'RF23M8090SG', 'RF28T5001SG'], dtype=object)

In [10]:
from openai import OpenAI
import openai

In [11]:
import os
from dotenv import load_dotenv
from pathlib import Path

env_path = Path('../.env')

# Load the .env file
load_dotenv(dotenv_path=env_path)

# Get the API key
openai_api_key = os.getenv("OPENAI_API_KEY")


#os.environ['OPENAI_API_KEY'] = openai_api_key
client = OpenAI()

In [32]:
def search(query):
    boost = {} #{'question':3.0, 'section':.5}
    knowledge_results = index.search(query=query, 
                                     filter_dict={}, #{'course':'data-engineering-zoomcamp'},
                                     boost_dict=boost, num_results=10)
    return knowledge_results

In [36]:
entry_template = '''
equipment_name: {equipment_name},
maker: {maker},
type_of_maintenance: {type_of_maintenance},
frequency: {frequency},
parts_required: {parts_required}
'''.strip()
def create_prompt(knowledge_results, query):
    context=''
    for doc in knowledge_results:
        context = context + entry_template.format(**doc) + '\n\n'
    prompt = f"""
    You're a appliance repair assistant. Answer the user QUESTION based on CONTEXT - the documents retrieved from our database. 
    Only use the facts from the CONTEXT. If the CONTEXT doesn't contan the answer, return "NONE"
                
    QUESTION: {question}
                
    CONTEXT:
                
    {context}
    """.strip()
    prompt = prompt.format(question=query, context=context)
    return prompt
    

In [33]:
def llm(prompt, client, model='gpt-3.5-turbo'):
    response = client.chat.completions.create(
        model=model,    
        messages=[{"role": "user", "content": prompt}]
    )
    return (response.choices[0].message.content)

In [34]:
def rag(query, client):
    knowledge_results = search(query)
    prompt = create_prompt(knowledge_results, query)
    return llm(prompt, client)

In [114]:
rag(question, client)

'For the Samsung Dishwasher model DW80R5061US, the annual maintenance includes checking for error codes, and no parts are required.'

In [57]:
#https://www.datacamp.com/tutorial/chromadb-tutorial-step-by-step-guide
#https://www.datacamp.com/tutorial/introduction-to-text-embeddings-with-the-open-ai-api

In [61]:
import chromadb
from chromadb.config import Settings


client = chromadb.Client()

**Evaluation**

In [61]:
prompt_template = """
You emulate a technician who's using our assistant app.
Formulate 5 questions this technician might ask based on a FAQ record. The record
should contain the answer to the questions, and the questions should be complete and not too short.
If possible, use as fewer words as possible from the record. 

The record:
id:{id}
equipment_name: {equipment_name}
maker: {maker}
type_of_maintenance: {type_of_maintenance}
frequency: {frequency}
parts_required: {parts_required}

Provide the output in parsable JSON without using code blocks, ensuring all keys are properly quoted:

{{"Questions": ["question1", "question2", ..., "question5"]}}
After generating the JSON, please verify that all keys and string values are properly enclosed in double quotes.
""".strip()

In [13]:
documents[0]

{'id': '0',
 'equipment_type': 'Dishwasher',
 'equipment_name': 'Series 6 SMS6ZCI48E',
 'maker': 'Bosch',
 'type_of_maintenance': 'Clean filter',
 'frequency': 'Monthly',
 'parts_required': 'None'}

In [25]:
print(prompt_template.format(**documents[0]))

You emulate a technician who's using our assistant app.
Formulate 5 questions this technician might ask based on a FAQ record. The record
should contain the answer to the questions, and the questions should be complete and not too short.
If possible, use as fewer words as possible from the record. 

The record:
id:0
equipment_name: Series 6 SMS6ZCI48E
maker: Bosch
type_of_maintenance: Clean filter
frequency: Monthly
parts_required: None

Provide the output in parsable JSON without using code blocks:

{"Questions": ["question1", "question2", ..., "question5"]}


In [27]:
print(llm(prompt_template.format(**documents[0]), client))

{
  "Questions": [
    "What equipment name requires clean filter maintenance monthly with no parts required?",
    "Which maker produces the Series 6 SMS6ZCI48E equipment?",
    "How often should the filter be cleaned in the maintenance of the Series 6 SMS6ZCI48E equipment?",
    "Are any parts required for the clean filter maintenance of the Series 6 SMS6ZCI48E equipment?",
    "What type of maintenance is required for the Series 6 SMS6ZCI48E equipment?"
  ]
}


In [28]:
import json

In [29]:
llm_reply  = json.loads(llm(prompt_template.format(**documents[0]), client))

In [31]:
print(llm_reply)

{'Questions': ['What is the recommended maintenance for the Series 6 SMS6ZCI48E?', 'How often should the filter be cleaned on the Series 6 SMS6ZCI48E?', 'Does the maintenance of the Series 6 SMS6ZCI48E require any additional parts?', 'Who is the manufacturer of the Series 6 SMS6ZCI48E?', 'What type of equipment is the Series 6 SMS6ZCI48E?']}


In [62]:
from tqdm.auto import tqdm

In [37]:

def generate_questions(document, client):
        prompt = prompt_template.format(**document)
        answer = llm(prompt, client)        
        return answer
        

In [63]:
results = {}
for doc in tqdm(documents): 
    id = doc['id']
    if id in results:
        continue

    questions_raw = generate_questions(doc, client)
    questions = json.loads(questions_raw)
    results[id] = questions['Questions']

  0%|          | 0/352 [00:00<?, ?it/s]

In [64]:
results['0']

['What is the recommended frequency for cleaning the filter of a Bosch Series 6 SMS6ZCI48E?',
 'Are any parts required for the maintenance of a Bosch Series 6 SMS6ZCI48E filter?',
 'Which brand manufactures the Bosch Series 6 SMS6ZCI48E equipment?',
 'What type of maintenance is recommended for the Bosch Series 6 SMS6ZCI48E equipment?',
 'What is the equipment name of the Bosch model SMS6ZCI48E?']

In [65]:
final_results = []

for doc_id, questions in results.items():
    for q in questions:
        final_results.append((doc_id, q))

In [66]:
final_results[0]

('0',
 'What is the recommended frequency for cleaning the filter of a Bosch Series 6 SMS6ZCI48E?')

In [67]:
df_results = pd.DataFrame(final_results, columns=['id', 'question'])

In [99]:
df_results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1756 entries, 0 to 1755
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        1756 non-null   object
 1   question  1756 non-null   object
dtypes: object(2)
memory usage: 27.6+ KB


In [105]:
df_results['id'] = df_results['id'].astype(str)

In [106]:
df_results.head()

,id,question
0,0,What is the recommended frequency for cleaning...
1,0,Are any parts required for the maintenance of ...
2,0,Which brand manufactures the Bosch Series 6 SM...
3,0,What type of maintenance is recommended for th...
4,0,What is the equipment name of the Bosch model ...


In [107]:
import csv
df_results.to_csv('../data/ground-truth-retrieval.csv', index=False, quoting=csv.QUOTE_NONNUMERIC, quotechar='"')

**Retreival Evaluation**

In [13]:
df_question = pd.read_csv('../data/ground-truth-retrieval.csv')

In [14]:
df_question.head()

,id,question
0,0,What is the recommended frequency for cleaning...
1,0,Are any parts required for the maintenance of ...
2,0,Which brand manufactures the Bosch Series 6 SM...
3,0,What type of maintenance is recommended for th...
4,0,What is the equipment name of the Bosch model ...


In [15]:
df_question['id'] = df_question['id'].astype(str)

In [16]:
df_question.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1756 entries, 0 to 1755
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        1756 non-null   object
 1   question  1756 non-null   object
dtypes: object(2)
memory usage: 27.6+ KB


In [75]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [76]:
def minsearch_search(query):
    boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [141]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    #for q in tqdm(ground_truth):
    for q in tqdm(ground_truth):
        
        doc_id = q['id']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        ''' print('query', q)
        print('id', q['id'])
        print(results)
        print('type relevance', )
        print(relevance)
        break'''
        relevance_total.append(relevance)
        
    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [18]:
ground_truth = df_question.to_dict(orient='records')

In [19]:
ground_truth[0]

{'id': '0',
 'question': 'What is the recommended frequency for cleaning the filter of a Bosch Series 6 SMS6ZCI48E?'}

In [24]:
from tqdm.auto import tqdm


In [122]:
evaluate(ground_truth, lambda q: search(q['question']))

  0%|          | 0/1756 [00:00<?, ?it/s]

{'hit_rate': 0.8553530751708428, 'mrr': 0.6823731333839534}

In [149]:
#!pip install hyperopt

## Finding the best parameters

In [126]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [127]:
len(df_question)

1756

In [129]:
df_validation = df_question[:int(len(df_question)/2)]
df_test = df_question[int(len(df_question)/2):]

In [130]:
import random

def simple_optimize(param_ranges, objective_function, n_iterations=10):
    best_params = None
    best_score = float('-inf')  # Assuming we're minimizing. Use float('-inf') if maximizing.

    for _ in range(n_iterations):
        # Generate random parameters
        current_params = {}
        for param, (min_val, max_val) in param_ranges.items():
            if isinstance(min_val, int) and isinstance(max_val, int):
                current_params[param] = random.randint(min_val, max_val)
            else:
                current_params[param] = random.uniform(min_val, max_val)
        
        # Evaluate the objective function
        current_score = objective_function(current_params)
        
        # Update best if current is better
        if current_score > best_score:  # Change to > if maximizing
            best_score = current_score
            best_params = current_params
    
    return best_params, best_score

In [131]:
gt_val = df_validation.to_dict(orient='records')

In [132]:
def minsearch_search(query, boost=None):
    if boost is None:
        boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [136]:
param_ranges = {
    'equipment_name': (0.0, 3.0),
    'maker': (0.0, 3.0),
    'type_of_maintenance': (0.0, 3.0),
    'frequency': (0.0, 3.0),
    'parts_required': (0.0, 3.0),
}

In [142]:
def objective(boost_params):
    def search_function(q):
        return minsearch_search(q['question'], boost_params)

    results = evaluate(gt_val, search_function)
    return results['mrr']

In [143]:
simple_optimize(param_ranges, objective, n_iterations=20)

  0%|          | 0/878 [00:00<?, ?it/s]

  0%|          | 0/878 [00:00<?, ?it/s]

  0%|          | 0/878 [00:00<?, ?it/s]

  0%|          | 0/878 [00:00<?, ?it/s]

  0%|          | 0/878 [00:00<?, ?it/s]

...

  0%|          | 0/878 [00:00<?, ?it/s]

  0%|          | 0/878 [00:00<?, ?it/s]

  0%|          | 0/878 [00:00<?, ?it/s]

  0%|          | 0/878 [00:00<?, ?it/s]

({'equipment_name': 2.2286308944548905,
  'maker': 1.3065317502078608,
  'type_of_maintenance': 2.7847555728963282,
  'frequency': 0.4028772934987217,
  'parts_required': 0.4007776344493833},
 0.7214498137903603)

In [147]:
def minsearch_improved(query):
    boost = {
        'equipment_name': 2.2286308944548905,
        'maker': 1.3065317502078608,
        'type_of_maintenance': 2.7847555728963282,
        'frequency': 0.4028772934987217,
        'parts_required': 0.4007776344493833
    }
    
    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [148]:
evaluate(ground_truth, lambda q: minsearch_improved(q['question']))

  0%|          | 0/1756 [00:00<?, ?it/s]

{'hit_rate': 0.8490888382687927, 'mrr': 0.7397254763712625}

## Evaluation, uusing LLM as Judge

In [17]:
prompt2_template = """
You are an expert evaluator for a Retrieval-Augmented Generation (RAG) system.
Your task is to analyze the relevance of the generated answer to the given question.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: {question}
Generated Answer: {answer_llm}

Please analyze the content and context of the generated answer in relation to the question
and provide your evaluation in parsable JSON without using code blocks:

{{
  "Relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT",
  "Explanation": "[Provide a brief explanation for your evaluation]"
}}
""".strip()

In [153]:
len(ground_truth)

1756

In [25]:
record = ground_truth[0]
record

{'id': '0',
 'question': 'What is the recommended frequency for cleaning the filter of a Bosch Series 6 SMS6ZCI48E?'}

In [167]:
question = record['question']
answer_llm = rag(question, client)

In [170]:
print(prompt2_template.format(question=question,answer_llm = answer_llm))

You are an expert evaluator for a Retrieval-Augmented Generation (RAG) system.
Your task is to analyze the relevance of the generated answer to the given question.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: What is the recommended frequency for cleaning the filter of a Bosch Series 6 SMS6ZCI48E?
Generated Answer: The recommended frequency for cleaning the filter of a Bosch Series 6 SMS6ZCI48E is monthly.

Please analyze the content and context of the generated answer in relation to the question
and provide your evaluation in parsable JSON without using code blocks:

{
  "Relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT",
  "Explanation": "[Provide a brief explanation for your evaluation]"
}


In [166]:
question

'What is the recommended frequency for cleaning the filter of a Bosch Series 6 SMS6ZCI48E?'

In [172]:
llm(question, client)

'The recommended frequency for cleaning the filter of a Bosch Series 6 SMS6ZCI48E dishwasher is every 2-3 months, or more frequently if you notice a decrease in performance or efficiency. Regular maintenance of the filter will help ensure optimal cleaning results and prolong the lifespan of your dishwasher.'

In [180]:
evaluations = []

for record in ground_truth:
    question = record['question']
    answer_llm = rag(question, client)

    prompt = prompt2_template.format(
        question=question,
        answer_llm=answer_llm
    )

    evaluation = llm(prompt, client)
    evaluation = json.loads(evaluation)

    evaluations.append((record, answer_llm, evaluation))

In [181]:
evaluations[0]

({'id': '0',
  'question': 'What is the recommended frequency for cleaning the filter of a Bosch Series 6 SMS6ZCI48E?'},
 'The recommended frequency for cleaning the filter of a Bosch Series 6 SMS6ZCI48E is monthly.',
 {'Relevance': 'RELEVANT',
  'Explanation': 'The generated answer directly addresses the question by providing the recommended frequency for cleaning the filter of a Bosch Series 6 SMS6ZCI48E, which is monthly. This answer is relevant as it provides a specific and accurate response to the question.'})

In [ ]:
gpt-3.5-turbo

In [182]:
df_eval = pd.DataFrame(evaluations, columns=['record', 'answer', 'evaluation'])

df_eval['id'] = df_eval.record.apply(lambda d: d['id'])
df_eval['question'] = df_eval.record.apply(lambda d: d['question'])

df_eval['relevance'] = df_eval.evaluation.apply(lambda d: d['Relevance'])
df_eval['explanation'] = df_eval.evaluation.apply(lambda d: d['Explanation'])

del df_eval['record']
del df_eval['evaluation']

In [183]:
df_eval.relevance.value_counts(normalize=True)

relevance
RELEVANT           0.428815
PARTLY_RELEVANT    0.378132
NON_RELEVANT       0.192483
Non_Relevant       0.000569
Name: proportion, dtype: float64

In [184]:
df_eval[df_eval.relevance == 'NON_RELEVANT']

,answer,id,question,relevance,explanation
18,As needed,3,How often should rinse aid be replaced for G 7...,NON_RELEVANT,The generated answer 'As needed' does not dire...
19,The parts required for replacing rinse aid on ...,3,What parts are required for replacing rinse ai...,NON_RELEVANT,The generated answer does not provide a list o...
23,PARTS REQUIRED: Soft cloth,4,What parts are required for cleaning the door ...,NON_RELEVANT,The generated answer only mentions a soft clot...
31,LG,6,Who is the manufacturer of the LDP6810SS equip...,NON_RELEVANT,The generated answer 'LG' does not directly ad...
42,"For the FDPC4660AS equipment, the recommended ...",8,3. What type of maintenance is recommended for...,NON_RELEVANT,The generated answer provides specific instruc...
...,...,...,...,...,...
1732,The parts required for the maintenance of the ...,347,What parts are required for the maintenance of...,NON_RELEVANT,The generated answer only mentions a vinegar s...
1733,The equipment name for the maintenance task in...,347,What is the equipment name for the maintenance...,NON_RELEVANT,The generated answer does not directly address...
1747,Soft cloth.,350,What equipment is required to clean the conden...,NON_RELEVANT,The generated answer 'Soft cloth' is not relev...
1750,The parts required for cleaning the condenser ...,350,What parts are required for cleaning the conde...,NON_RELEVANT,The generated answer only mentions a soft clot...


In [196]:
df_eval.loc[1750]

answer         The parts required for cleaning the condenser ...
id                                                           350
question       What parts are required for cleaning the conde...
relevance                                           NON_RELEVANT
explanation    The generated answer only mentions a soft clot...
Name: 1750, dtype: object

In [198]:
df_eval.loc[1750]['question']

'What parts are required for cleaning the condenser fan blade on the MFI2269FRZ?'

In [197]:
df_eval.loc[1750]['explanation']

'The generated answer only mentions a soft cloth as the part required for cleaning the condenser fan blade, which is not accurate. The question asks for all the parts required, not just one.'

In [185]:
df_eval.to_csv('../data/rag-eval-gpt-3.5-turbo_all.csv', index=False)

In [26]:
df_sample = df_question.sample(n=200, random_state=1)

In [27]:
sample = df_sample.to_dict(orient='records')

In [48]:
prompt3_template = """
You are an expert evaluator for a Retrieval-Augmented Generation (RAG) system.
Your task is to analyze the relevance of the generated answer to the given question.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Please classify valid company names as "RELEVANT" answer.
Please consider part or parts required as "RELEVANT".
Please consider answer "RELEVANT" as long answer doesn't conflict with technical facts about the machine.

Here is the data for evaluation:

Question: {question}
Generated Answer: {answer_llm}

Please analyze the content and context of the generated answer in relation to the question
and provide your evaluation in parsable JSON without using code blocks:

{{
  "Relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT",
  "Explanation": "[Provide a brief explanation for your evaluation]"
}}
""".strip()

In [49]:
evaluations_sample = []

for record in sample:
    question = record['question']
    answer_llm = rag(question, client)

    prompt = prompt3_template.format(
        question=question,
        answer_llm=answer_llm
    )

    evaluation = llm(prompt, client)
    evaluation = json.loads(evaluation)

    evaluations_sample.append((record, answer_llm, evaluation))

In [50]:
df_eval_sample = pd.DataFrame(evaluations_sample, columns=['record', 'answer', 'evaluation'])

df_eval_sample['id'] = df_eval_sample.record.apply(lambda d: d['id'])
df_eval_sample['question'] = df_eval_sample.record.apply(lambda d: d['question'])

df_eval_sample['relevance'] = df_eval_sample.evaluation.apply(lambda d: d['Relevance'])
df_eval_sample['explanation'] = df_eval_sample.evaluation.apply(lambda d: d['Explanation'])

del df_eval_sample['record']
del df_eval_sample['evaluation']

In [51]:
df_eval_sample.relevance.value_counts(normalize=True)

relevance
PARTLY_RELEVANT    0.43
RELEVANT           0.34
NON_RELEVANT       0.23
Name: proportion, dtype: float64

In [55]:
df_eval_sample[df_eval_sample.relevance == 'NON_RELEVANT'].head()

,answer,id,question,relevance,explanation
12,The equipment that needs to be checked annuall...,301,What equipment needs to be checked annually?,NON_RELEVANT,The generated answer lists specific models of ...
13,Mild detergent,262,What parts are required for cleaning the exter...,NON_RELEVANT,The generated answer 'Mild detergent' does not...
19,The parts required for maintenance of the WRS5...,301,What parts are required for maintenance of WRS...,NON_RELEVANT,"The answer provided, stating that a Multimeter..."
21,The type of maintenance recommended for the Bo...,20,What type of maintenance is recommended for th...,NON_RELEVANT,The answer provided does not directly address ...
23,NONE. Inspecting the control board does not re...,59,Does inspecting the control board require any ...,NON_RELEVANT,The answer states that inspecting the control ...


In [54]:
df_eval_sample[df_eval_sample.relevance == 'NON_RELEVANT'].loc[12]['explanation']

'The generated answer lists specific models of appliances from different brands without mentioning any general equipment that needs to be checked annually. It does not directly address the question asked.'

In [56]:
df_eval_sample[df_eval_sample.relevance == 'NON_RELEVANT'].loc[12]['answer']

'The equipment that needs to be checked annually are the G 4228 SCU by Miele for inspecting spray arm seals and the LDP6810SS by LG for checking the water inlet valve.'

In [57]:
evaluations_sample_4mini = []

for record in sample:
    question = record['question']
    answer_llm = rag(question, client)

    prompt = prompt3_template.format(
        question=question,
        answer_llm=answer_llm
    )

    evaluation = llm(prompt, client, model='gpt-4o-mini')
    evaluation = json.loads(evaluation)

    evaluations_sample_4mini.append((record, answer_llm, evaluation))

In [58]:
df_eval_sample4mini = pd.DataFrame(evaluations_sample_4mini, columns=['record', 'answer', 'evaluation'])

df_eval_sample4mini['id'] = df_eval_sample4mini.record.apply(lambda d: d['id'])
df_eval_sample4mini['question'] = df_eval_sample4mini.record.apply(lambda d: d['question'])

df_eval_sample4mini['relevance'] = df_eval_sample4mini.evaluation.apply(lambda d: d['Relevance'])
df_eval_sample4mini['explanation'] = df_eval_sample4mini.evaluation.apply(lambda d: d['Explanation'])

del df_eval_sample4mini['record']
del df_eval_sample4mini['evaluation']

In [60]:
df_eval_sample4mini.relevance.value_counts(normalize=True)

relevance
RELEVANT           0.825
PARTLY_RELEVANT    0.110
NON_RELEVANT       0.065
Name: proportion, dtype: float64

## Ingestion

In [199]:
df_all = pd.read_csv('..\data\data_cleaned.csv')
df_all = df_all.replace({np.nan: 'None'})
df_all['id'] = df_all['id'].astype(str)
documents = df_all.to_dict(orient='records')
index = minsearch.Index(
    text_fields=['id','equipment_type', 'equipment_name', 'maker', 'type_of_maintenance', 'frequency',
       'parts_required'],
    keyword_fields=[id]
)
index.fit(documents)

In [200]:
#use search with boost paramters
def minsearch_improved(query):
    boost = {
        'equipment_name': 2.2286308944548905,
        'maker': 1.3065317502078608,
        'type_of_maintenance': 2.7847555728963282,
        'frequency': 0.4028772934987217,
        'parts_required': 0.4007776344493833
    }
    
    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results